In [1]:
# https://www.kaggle.com/mathormad/inceptionv3-baseline-lb-0-379/code

In [2]:
# !pip install iterative-stratification

In [3]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image
import cv2
from sklearn.utils import class_weight, shuffle
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold, MultilabelStratifiedShuffleSplit

import warnings
warnings.filterwarnings("ignore")
SIZE = 299

In [4]:
# https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric/notebook
import tensorflow as tf

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1) + (1 - K.mean(f1))

In [5]:
# Load dataset info
path_to_train = '../data/train/'
data = pd.read_csv('../data/train.csv')

In [6]:
data.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [7]:
train_dataset_info = []
for name, labels in zip(data['Id'], data['Target'].str.split(' ')):
    train_dataset_info.append({
        'path':os.path.join(path_to_train, name),
        'labels':np.array([int(label) for label in labels])})
train_dataset_info = np.array(train_dataset_info)

In [8]:
train_dataset_info

array([{'path': '../data/train/00070df0-bbc3-11e8-b2bc-ac1f6b6435d0', 'labels': array([16,  0])},
       {'path': '../data/train/000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0', 'labels': array([7, 1, 2, 0])},
       {'path': '../data/train/000a9596-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([5])},
       ...,
       {'path': '../data/train/fff189d8-bbab-11e8-b2ba-ac1f6b6435d0', 'labels': array([7])},
       {'path': '../data/train/fffdf7e0-bbc4-11e8-b2bc-ac1f6b6435d0', 'labels': array([25,  2, 21])},
       {'path': '../data/train/fffe0ffe-bbc0-11e8-b2bb-ac1f6b6435d0', 'labels': array([2, 0])}],
      dtype=object)

In [9]:
class data_generator:
    
    def create_train(dataset_info, batch_size, shape, augument=True):
        assert shape[2] == 3
        while True:
            dataset_info = shuffle(dataset_info)
            for start in range(0, len(dataset_info), batch_size):
                end = min(start + batch_size, len(dataset_info))
                batch_images = []
                X_train_batch = dataset_info[start:end]
                batch_labels = np.zeros((len(X_train_batch), 28))
                for i in range(len(X_train_batch)):
                    image = data_generator.load_image(
                        X_train_batch[i]['path'], shape)   
                    if augument:
                        image = data_generator.augment(image)
                    batch_images.append(image/255.)
                    batch_labels[i][X_train_batch[i]['labels']] = 1
                yield np.array(batch_images, np.float32), batch_labels

    def load_image(path, shape):
        image_red_ch = Image.open(path+'_red.png')
        image_yellow_ch = Image.open(path+'_yellow.png')
        image_green_ch = Image.open(path+'_green.png')
        image_blue_ch = Image.open(path+'_blue.png')
        image = np.stack((
            np.array(image_red_ch),
            np.array(image_green_ch), 
            np.array(image_blue_ch)), -1)
        w, h = 512, 512
        zero_data = np.zeros((h, w), dtype=np.uint8)
#         image2 = np.stack((
#             np.array(image_yellow_ch),
#             zero_data, zero_data), -1)
#         print(image1.shape, image2.shape)
#         image = np.vstack((image1, image2))
        image = cv2.resize(image, (shape[0], shape[1]))
        return image

    def augment(image):
        augment_img = iaa.Sequential([
            iaa.OneOf([
                iaa.Affine(rotate=0),
                iaa.Affine(rotate=90),
                iaa.Affine(rotate=180),
                iaa.Affine(rotate=270),
                iaa.Fliplr(0.5),
                iaa.Flipud(0.5),
            ])], random_order=True)

        image_aug = augment_img.augment_image(image)
        return image_aug


In [10]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model

Using TensorFlow backend.


In [11]:
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    base_model = InceptionV3(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape)
    bn = BatchNormalization()(input_tensor)
    x = base_model(bn)
    x = Conv2D(32, kernel_size=(1,1), activation='relu')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    
    return model

In [12]:
import tensorflow as tf
from tensorflow.python.ops import array_ops

# https://github.com/ailias/Focal-Loss-implement-on-Tensorflow/blob/master/focal_loss.py
def focal_loss_org(prediction_tensor, target_tensor, weights=None, alpha=0.25, gamma=2):
    r"""Compute focal loss for predictions.
        Multi-labels Focal loss formula:
            FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log(1-p)
                 ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
    Args:
     prediction_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing the predicted logits for each class
     target_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing one-hot encoded classification targets
     weights: A float tensor of shape [batch_size, num_anchors]
     alpha: A scalar tensor for focal loss alpha hyper-parameter
     gamma: A scalar tensor for focal loss gamma hyper-parameter
    Returns:
        loss: A (scalar) tensor representing the value of the loss function
    """
    sigmoid_p = tf.nn.sigmoid(prediction_tensor)
    zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)
    
    # For poitive prediction, only need consider front part loss, back part is 0;
    # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
    # For negative prediction, only need consider back part loss, front part is 0;
    # target_tensor > zeros <=> z=1, so negative coefficient = 0.
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)


In [13]:
def focal_loss(weights=None, alpha=0.25, gamma=2):
    def focal_loss_my(target_tensor, prediction_tensor, ):
        r"""Compute focal loss for predictions.
            Multi-labels Focal loss formula:
                FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log(1-p)
                     ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
        Args:
         prediction_tensor: A float tensor of shape [batch_size, num_anchors,
            num_classes] representing the predicted logits for each class
         target_tensor: A float tensor of shape [batch_size, num_anchors,
            num_classes] representing one-hot encoded classification targets
         weights: A float tensor of shape [batch_size, num_anchors]
         alpha: A scalar tensor for focal loss alpha hyper-parameter
         gamma: A scalar tensor for focal loss gamma hyper-parameter
        Returns:
            loss: A (scalar) tensor representing the value of the loss function
        """
        sigmoid_p = tf.nn.sigmoid(prediction_tensor)
        zeros = array_ops.zeros_like(sigmoid_p, dtype=sigmoid_p.dtype)

        # For poitive prediction, only need consider front part loss, back part is 0;
        # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
        pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)

        # For negative prediction, only need consider back part loss, front part is 0;
        # target_tensor > zeros <=> z=1, so negative coefficient = 0.
        neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
        per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                              - (1 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
        return tf.reduce_sum(per_entry_cross_ent)
#         return K.mean(K.binary_crossentropy(target_tensor, prediction_tensor), axis=-1) + tf.reduce_sum(per_entry_cross_ent)
    return focal_loss_my

In [14]:
def focal_loss_fixed(y_true, y_pred):
    gamma = 2.
    alpha = 0.25
    print(y_pred)
    print(y_true)
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

#     pt_1 = K.clip(pt_1, 1e-3, .999)
#     pt_0 = K.clip(pt_0, 1e-3, .999)

    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))


In [15]:
# def focal_loss(gamma=2., alpha=.25):
#     def focal_loss_fixed(y_true, y_pred):
#         pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
#         pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

#         pt_1 = K.clip(pt_1, 1e-3, .999)
#         pt_0 = K.clip(pt_0, 1e-3, .999)

#         return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
#     return focal_loss_fixed

In [16]:
# create callbacks list
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split



In [17]:
train_dataset_info.shape

(31072,)

In [18]:
n_classes = 28
y_train = np.zeros((train_dataset_info.shape[0], n_classes))
print(y_train.shape)

idx = 0
for labels in tqdm(data['Target'].str.split(' ')):
#     print(labels)
    for label in labels:
        y_train[idx][int(label)] = 1
    idx += 1

100%|██████████| 31072/31072 [00:00<00:00, 625442.90it/s]

(31072, 28)


In [21]:

# split data into train, valid
indexes = np.arange(train_dataset_info.shape[0])
# np.random.shuffle(indexes)
# train_indexes, valid_indexes = train_test_split(indexes, test_size=0.15, random_state=8)
n_splits = 5
# kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)
kf = MultilabelStratifiedShuffleSplit(n_splits=5, random_state=42)
submit = pd.read_csv('../data/sample_submission.csv')

# train_generator = data_generator.create_train(
#     train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
# validation_generator = data_generator.create_train(
#     train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=False)

oof_class_preds = np.zeros((train_dataset_info.shape[0], 28))
sub_class_preds = np.zeros((submit.shape[0], 28))

oof_class_preds = np.load('../cache/oof_class_preds-14.npy')
sub_class_preds = np.load('../cache/sub_class_preds-14.npy')
                          
fold_ = 0
epochs = 10; batch_size = 16
my_fold = 0
for train_indexes, valid_indexes in kf.split(indexes, y_train):
    print(len(train_indexes), len(valid_indexes))
    if my_fold < 2:
        my_fold += 1
        continue
    checkpoint = ModelCheckpoint('../cache/InceptionV3.h5', monitor='val_loss', verbose=1, 
                                 save_best_only=True, mode='min', save_weights_only = True)
    reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, 
                                       verbose=1, mode='auto', epsilon=0.0001)
    early = EarlyStopping(monitor="val_loss", 
                          mode="min", 
                          patience=6)
    callbacks_list = [checkpoint, early, reduceLROnPlat]


    _preds = []
    # create train and valid datagens
    train_generator = data_generator.create_train(
        train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
    validation_generator = data_generator.create_train(
        train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=False)

    # warm up model
    model = create_model(
        input_shape=(SIZE,SIZE,3), 
        n_out=28)

    for layer in model.layers:
        layer.trainable = False
    model.layers[-1].trainable = True
    model.layers[-2].trainable = True
    model.layers[-3].trainable = True
    model.layers[-4].trainable = True
    model.layers[-5].trainable = True
    model.layers[-6].trainable = True
    
    model.compile(
        loss=f1_loss, 
        optimizer=Adam(1e-03),
        metrics=[f1])
#     model.summary()

    model.fit_generator(
        train_generator,
        steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
        validation_data=validation_generator,
        validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
        epochs=2, 
        verbose=1)
    
    # train all layers
    epochs=120
    for layer in model.layers:
        layer.trainable = True
    model.compile(loss=f1_loss,
                optimizer=Adam(lr=1e-4),
                metrics=[f1])
    model.fit_generator(
        train_generator,
        steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
        validation_data=validation_generator,
        validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
        epochs=epochs, 
        verbose=1,
        callbacks=callbacks_list)
    
    for idx in tqdm(valid_indexes):
        item = train_dataset_info[idx]
        path = item['path']
        labels = item['labels']
        image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
        score_predict = model.predict(image[np.newaxis])[0]
        oof_class_preds[idx] = score_predict
        np.save('../cache/oof_class_preds-14.npy', oof_class_preds)
        
    for idx, name in tqdm(enumerate(submit['Id'])):
        path = os.path.join('../data/test/', name)
        image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
        score_predict = model.predict(image[np.newaxis])[0]
        sub_class_preds[idx] += score_predict
        np.save('../cache/sub_class_preds-14.npy', sub_class_preds)
    fold_ += 1
sub_class_preds /= n_splits

27958 3114
27973 3099
27969 3103
Epoch 1/2
1749/1749 [==============================] - 1523s 871ms/step - loss: 1.1180 - f1: 0.0402 - val_loss: 1.1985 - val_f1: 0.0329
Epoch 2/2
1749/1749 [==============================] - 324s 186ms/step - loss: 1.1038 - f1: 0.0513 - val_loss: 1.1325 - val_f1: 0.0272
Epoch 1/120
1749/1749 [==============================] - 414s 236ms/step - loss: 1.0478 - f1: 0.1068 - val_loss: 0.9834 - val_f1: 0.1947

Epoch 00001: val_loss improved from inf to 0.98341, saving model to ../cache/InceptionV3.h5
Epoch 2/120
1749/1749 [==============================] - 399s 228ms/step - loss: 0.9747 - f1: 0.1745 - val_loss: 0.9182 - val_f1: 0.2328

Epoch 00002: val_loss improved from 0.98341 to 0.91816, saving model to ../cache/InceptionV3.h5
Epoch 3/120
1749/1749 [==============================] - 397s 227ms/step - loss: 0.9331 - f1: 0.2088 - val_loss: 0.9284 - val_f1: 0.2466

Epoch 00003: val_loss did not improve from 0.91816
Epoch 4/120
1749/1749 [====================

  0%|          | 0/3103 [00:00<?, ?it/s]


Epoch 00022: val_loss did not improve from 0.72914

Epoch 00022: ReduceLROnPlateau reducing learning rate to 9.999999974752428e-08.


100%|██████████| 3103/3103 [02:45<00:00, 18.71it/s]
11702it [20:02,  9.73it/s]


27995 3077
Epoch 1/2
1750/1750 [==============================] - 333s 190ms/step - loss: 1.1175 - f1: 0.0407 - val_loss: 1.1790 - val_f1: 0.0365
Epoch 2/2
1750/1750 [==============================] - 324s 185ms/step - loss: 1.1043 - f1: 0.0514 - val_loss: 1.1739 - val_f1: 0.0305
Epoch 1/120
1750/1750 [==============================] - 415s 237ms/step - loss: 1.0517 - f1: 0.1021 - val_loss: 1.0347 - val_f1: 0.1529

Epoch 00001: val_loss improved from inf to 1.03473, saving model to ../cache/InceptionV3.h5
Epoch 2/120
1750/1750 [==============================] - 397s 227ms/step - loss: 0.9839 - f1: 0.1652 - val_loss: 0.9320 - val_f1: 0.2228

Epoch 00002: val_loss improved from 1.03473 to 0.93204, saving model to ../cache/InceptionV3.h5
Epoch 3/120
1750/1750 [==============================] - 395s 226ms/step - loss: 0.9394 - f1: 0.2026 - val_loss: 0.9169 - val_f1: 0.2498

Epoch 00003: val_loss improved from 0.93204 to 0.91691, saving model to ../cache/InceptionV3.h5
Epoch 4/120
1750/1750

...

100%|██████████| 3077/3077 [02:42<00:00, 18.90it/s]
11702it [08:10, 24.53it/s]


27928 3144
Epoch 1/2
1746/1746 [==============================] - 333s 190ms/step - loss: 1.1190 - f1: 0.0394 - val_loss: 1.1304 - val_f1: 0.0292
Epoch 2/2
1746/1746 [==============================] - 322s 185ms/step - loss: 1.1049 - f1: 0.0502 - val_loss: 1.1549 - val_f1: 0.0227
Epoch 1/120
1746/1746 [==============================] - 405s 232ms/step - loss: 1.0552 - f1: 0.0992 - val_loss: 1.0144 - val_f1: 0.1589

Epoch 00001: val_loss improved from inf to 1.01443, saving model to ../cache/InceptionV3.h5
Epoch 2/120
1746/1746 [==============================] - 383s 219ms/step - loss: 0.9849 - f1: 0.1632 - val_loss: 0.9488 - val_f1: 0.2134

Epoch 00002: val_loss improved from 1.01443 to 0.94883, saving model to ../cache/InceptionV3.h5
Epoch 3/120
1746/1746 [==============================] - 382s 219ms/step - loss: 0.9362 - f1: 0.2067 - val_loss: 0.9174 - val_f1: 0.2545

Epoch 00003: val_loss improved from 0.94883 to 0.91744, saving model to ../cache/InceptionV3.h5
Epoch 4/120
1746/1746

  0%|          | 0/3144 [00:00<?, ?it/s]


Epoch 00029: val_loss did not improve from 0.73207

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1.0000000116860975e-08.


100%|██████████| 3144/3144 [03:04<00:00, 20.41it/s]
11702it [08:20, 23.40it/s]


In [22]:
# 27958 3114
# Epoch 1/2
# 1748/1748 [==============================] - 1797s 1s/step - loss: 1.1195 - f1: 0.0376 - val_loss: 1.1698 - val_f1: 0.0320
# Epoch 2/2
# 1748/1748 [==============================] - 325s 186ms/step - loss: 1.1067 - f1: 0.0476 - val_loss: 1.1549 - val_f1: 0.0308
# Epoch 1/120
# 1748/1748 [==============================] - 413s 236ms/step - loss: 1.0568 - f1: 0.0977 - val_loss: 0.9852 - val_f1: 0.1725

# Epoch 00001: val_loss improved from inf to 0.98515, saving model to ../cache/InceptionV3.h5
# Epoch 2/120
# 1748/1748 [==============================] - 396s 227ms/step - loss: 0.9804 - f1: 0.1686 - val_loss: 0.9657 - val_f1: 0.2039

# Epoch 00002: val_loss improved from 0.98515 to 0.96568, saving model to ../cache/InceptionV3.h5
# Epoch 3/120
# 1748/1748 [==============================] - 401s 229ms/step - loss: 0.9326 - f1: 0.2086 - val_loss: 0.8926 - val_f1: 0.2645

# Epoch 00003: val_loss improved from 0.96568 to 0.89257, saving model to ../cache/InceptionV3.h5
# Epoch 4/120
# 1748/1748 [==============================] - 403s 230ms/step - loss: 0.9055 - f1: 0.2302 - val_loss: 0.8653 - val_f1: 0.2878

# Epoch 00004: val_loss improved from 0.89257 to 0.86533, saving model to ../cache/InceptionV3.h5
# Epoch 5/120
# 1748/1748 [==============================] - 406s 232ms/step - loss: 0.8883 - f1: 0.2408 - val_loss: 0.8512 - val_f1: 0.3025

# Epoch 00005: val_loss improved from 0.86533 to 0.85124, saving model to ../cache/InceptionV3.h5
# Epoch 6/120
# 1748/1748 [==============================] - 407s 233ms/step - loss: 0.8755 - f1: 0.2495 - val_loss: 0.8227 - val_f1: 0.3191

# Epoch 00006: val_loss improved from 0.85124 to 0.82271, saving model to ../cache/InceptionV3.h5
# Epoch 7/120
# 1748/1748 [==============================] - 404s 231ms/step - loss: 0.8642 - f1: 0.2577 - val_loss: 0.8060 - val_f1: 0.3284

# Epoch 00007: val_loss improved from 0.82271 to 0.80597, saving model to ../cache/InceptionV3.h5
# Epoch 8/120
# 1748/1748 [==============================] - 404s 231ms/step - loss: 0.8521 - f1: 0.2657 - val_loss: 0.8056 - val_f1: 0.3273

# Epoch 00008: val_loss improved from 0.80597 to 0.80562, saving model to ../cache/InceptionV3.h5
# Epoch 9/120
# 1748/1748 [==============================] - 402s 230ms/step - loss: 0.8428 - f1: 0.2711 - val_loss: 0.7919 - val_f1: 0.3408

# Epoch 00009: val_loss improved from 0.80562 to 0.79187, saving model to ../cache/InceptionV3.h5
# Epoch 10/120
# 1748/1748 [==============================] - 399s 228ms/step - loss: 0.8353 - f1: 0.2757 - val_loss: 0.8004 - val_f1: 0.3347

# Epoch 00010: val_loss did not improve from 0.79187
# Epoch 11/120
# 1748/1748 [==============================] - 400s 229ms/step - loss: 0.8286 - f1: 0.2815 - val_loss: 0.7983 - val_f1: 0.3329

# Epoch 00011: val_loss did not improve from 0.79187
# Epoch 12/120
# 1748/1748 [==============================] - 396s 226ms/step - loss: 0.8188 - f1: 0.2875 - val_loss: 0.7782 - val_f1: 0.3509

# Epoch 00012: val_loss improved from 0.79187 to 0.77825, saving model to ../cache/InceptionV3.h5
# Epoch 13/120
# 1748/1748 [==============================] - 396s 226ms/step - loss: 0.8137 - f1: 0.2905 - val_loss: 0.7710 - val_f1: 0.3602

# Epoch 00013: val_loss improved from 0.77825 to 0.77102, saving model to ../cache/InceptionV3.h5
# Epoch 14/120
# 1748/1748 [==============================] - 396s 227ms/step - loss: 0.8078 - f1: 0.2949 - val_loss: 0.7753 - val_f1: 0.3552

# Epoch 00014: val_loss did not improve from 0.77102
# Epoch 15/120
# 1748/1748 [==============================] - 396s 226ms/step - loss: 0.8001 - f1: 0.2995 - val_loss: 0.7773 - val_f1: 0.3505

# Epoch 00015: val_loss did not improve from 0.77102
# Epoch 16/120
# 1748/1748 [==============================] - 395s 226ms/step - loss: 0.7972 - f1: 0.3004 - val_loss: 0.8064 - val_f1: 0.3335

# Epoch 00016: val_loss did not improve from 0.77102

# Epoch 00016: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
# Epoch 17/120
# 1748/1748 [==============================] - 396s 226ms/step - loss: 0.7710 - f1: 0.3174 - val_loss: 0.7301 - val_f1: 0.3866

# Epoch 00017: val_loss improved from 0.77102 to 0.73014, saving model to ../cache/InceptionV3.h5
# Epoch 18/120
# 1748/1748 [==============================] - 395s 226ms/step - loss: 0.7606 - f1: 0.3238 - val_loss: 0.7375 - val_f1: 0.3798

# Epoch 00018: val_loss did not improve from 0.73014
# Epoch 19/120
# 1748/1748 [==============================] - 396s 226ms/step - loss: 0.7575 - f1: 0.3253 - val_loss: 0.7297 - val_f1: 0.3862

# Epoch 00019: val_loss improved from 0.73014 to 0.72971, saving model to ../cache/InceptionV3.h5
# Epoch 20/120
# 1748/1748 [==============================] - 394s 225ms/step - loss: 0.7554 - f1: 0.3257 - val_loss: 0.7304 - val_f1: 0.3831

# Epoch 00020: val_loss did not improve from 0.72971
# Epoch 21/120
# 1748/1748 [==============================] - 395s 226ms/step - loss: 0.7496 - f1: 0.3294 - val_loss: 0.7343 - val_f1: 0.3803

# Epoch 00021: val_loss did not improve from 0.72971
# Epoch 22/120
# 1748/1748 [==============================] - 393s 225ms/step - loss: 0.7482 - f1: 0.3302 - val_loss: 0.7265 - val_f1: 0.3887

# Epoch 00022: val_loss improved from 0.72971 to 0.72649, saving model to ../cache/InceptionV3.h5
# Epoch 23/120
# 1748/1748 [==============================] - 394s 226ms/step - loss: 0.7444 - f1: 0.3328 - val_loss: 0.7339 - val_f1: 0.3818

# Epoch 00023: val_loss did not improve from 0.72649
# Epoch 24/120
# 1748/1748 [==============================] - 394s 225ms/step - loss: 0.7426 - f1: 0.3338 - val_loss: 0.7304 - val_f1: 0.3866

# Epoch 00024: val_loss did not improve from 0.72649
# Epoch 25/120
# 1748/1748 [==============================] - 394s 226ms/step - loss: 0.7398 - f1: 0.3354 - val_loss: 0.7324 - val_f1: 0.3816

# Epoch 00025: val_loss did not improve from 0.72649

# Epoch 00025: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.
# Epoch 26/120
# 1748/1748 [==============================] - 395s 226ms/step - loss: 0.7369 - f1: 0.3367 - val_loss: 0.7284 - val_f1: 0.3866

# Epoch 00026: val_loss did not improve from 0.72649
# Epoch 27/120
# 1748/1748 [==============================] - 394s 225ms/step - loss: 0.7358 - f1: 0.3378 - val_loss: 0.7321 - val_f1: 0.3835

# Epoch 00027: val_loss did not improve from 0.72649
# Epoch 28/120
# 1748/1748 [==============================] - 393s 225ms/step - loss: 0.7357 - f1: 0.3381 - val_loss: 0.7290 - val_f1: 0.3877
#   0%|          | 0/3114 [00:00<?, ?it/s]

# Epoch 00028: val_loss did not improve from 0.72649

# Epoch 00028: ReduceLROnPlateau reducing learning rate to 9.999999974752428e-08.
# 100%|██████████| 3114/3114 [02:42<00:00, 19.14it/s]
# 11702it [21:06,  9.24it/s]
# 27973 3099
# Epoch 1/2
# 1749/1749 [==============================] - 331s 190ms/step - loss: 1.1197 - f1: 0.0378 - val_loss: 1.1816 - val_f1: 0.0357
# Epoch 2/2
# 1749/1749 [==============================] - 319s 182ms/step - loss: 1.1067 - f1: 0.0478 - val_loss: 1.2044 - val_f1: 0.0347
# Epoch 1/120
# 1749/1749 [==============================] - 405s 232ms/step - loss: 1.0601 - f1: 0.0933 - val_loss: 0.9947 - val_f1: 0.1676

# Epoch 00001: val_loss improved from inf to 0.99467, saving model to ../cache/InceptionV3.h5
# Epoch 2/120
# 1749/1749 [==============================] - 384s 220ms/step - loss: 0.9880 - f1: 0.1613 - val_loss: 0.9376 - val_f1: 0.2233

# Epoch 00002: val_loss improved from 0.99467 to 0.93759, saving model to ../cache/InceptionV3.h5
# Epoch 3/120
# 1749/1749 [==============================] - 385s 220ms/step - loss: 0.9386 - f1: 0.2050 - val_loss: 0.8781 - val_f1: 0.2778

# Epoch 00003: val_loss improved from 0.93759 to 0.87811, saving model to ../cache/InceptionV3.h5
# Epoch 4/120
# 1749/1749 [==============================] - 385s 220ms/step - loss: 0.9109 - f1: 0.2256 - val_loss: 0.8430 - val_f1: 0.2990

# Epoch 00004: val_loss improved from 0.87811 to 0.84296, saving model to ../cache/InceptionV3.h5
# Epoch 5/120
# 1749/1749 [==============================] - 384s 220ms/step - loss: 0.8937 - f1: 0.2383 - val_loss: 0.8510 - val_f1: 0.2958

# Epoch 00005: val_loss did not improve from 0.84296
# Epoch 6/120
# 1749/1749 [==============================] - 384s 219ms/step - loss: 0.8799 - f1: 0.2472 - val_loss: 0.8430 - val_f1: 0.3046

# Epoch 00006: val_loss did not improve from 0.84296
# Epoch 7/120
# 1749/1749 [==============================] - 384s 219ms/step - loss: 0.8679 - f1: 0.2548 - val_loss: 0.8243 - val_f1: 0.3158

# Epoch 00007: val_loss improved from 0.84296 to 0.82426, saving model to ../cache/InceptionV3.h5
# Epoch 8/120
# 1749/1749 [==============================] - 383s 219ms/step - loss: 0.8553 - f1: 0.2641 - val_loss: 0.8559 - val_f1: 0.2988

# Epoch 00008: val_loss did not improve from 0.82426
# Epoch 9/120
# 1749/1749 [==============================] - 384s 219ms/step - loss: 0.8510 - f1: 0.2662 - val_loss: 0.8497 - val_f1: 0.3030

# Epoch 00009: val_loss did not improve from 0.82426
# Epoch 10/120
# 1749/1749 [==============================] - 382s 218ms/step - loss: 0.8410 - f1: 0.2731 - val_loss: 0.8813 - val_f1: 0.2890

# Epoch 00010: val_loss did not improve from 0.82426

# Epoch 00010: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
# Epoch 11/120
# 1749/1749 [==============================] - 382s 219ms/step - loss: 0.8165 - f1: 0.2874 - val_loss: 0.7563 - val_f1: 0.3650

# Epoch 00011: val_loss improved from 0.82426 to 0.75628, saving model to ../cache/InceptionV3.h5
# Epoch 12/120
# 1749/1749 [==============================] - 382s 218ms/step - loss: 0.8073 - f1: 0.2933 - val_loss: 0.7564 - val_f1: 0.3663

# Epoch 00012: val_loss did not improve from 0.75628
# Epoch 13/120
# 1749/1749 [==============================] - 383s 219ms/step - loss: 0.8035 - f1: 0.2963 - val_loss: 0.7528 - val_f1: 0.3692

# Epoch 00013: val_loss improved from 0.75628 to 0.75284, saving model to ../cache/InceptionV3.h5
# Epoch 14/120
# 1749/1749 [==============================] - 383s 219ms/step - loss: 0.7986 - f1: 0.2995 - val_loss: 0.7515 - val_f1: 0.3681

# Epoch 00014: val_loss improved from 0.75284 to 0.75147, saving model to ../cache/InceptionV3.h5
# Epoch 15/120
# 1749/1749 [==============================] - 382s 219ms/step - loss: 0.7943 - f1: 0.3026 - val_loss: 0.7538 - val_f1: 0.3696

# Epoch 00015: val_loss did not improve from 0.75147
# Epoch 16/120
# 1749/1749 [==============================] - 382s 219ms/step - loss: 0.7955 - f1: 0.3003 - val_loss: 0.7536 - val_f1: 0.3683

# Epoch 00016: val_loss did not improve from 0.75147
# Epoch 17/120
# 1749/1749 [==============================] - 384s 219ms/step - loss: 0.7889 - f1: 0.3054 - val_loss: 0.7508 - val_f1: 0.3676

# Epoch 00017: val_loss improved from 0.75147 to 0.75082, saving model to ../cache/InceptionV3.h5
# Epoch 18/120
# 1749/1749 [==============================] - 383s 219ms/step - loss: 0.7860 - f1: 0.3074 - val_loss: 0.7566 - val_f1: 0.3672

# Epoch 00018: val_loss did not improve from 0.75082
# Epoch 19/120
# 1749/1749 [==============================] - 383s 219ms/step - loss: 0.7832 - f1: 0.3099 - val_loss: 0.7517 - val_f1: 0.3718

# Epoch 00019: val_loss did not improve from 0.75082
# Epoch 20/120
# 1749/1749 [==============================] - 382s 219ms/step - loss: 0.7794 - f1: 0.3112 - val_loss: 0.7510 - val_f1: 0.3683

# Epoch 00020: val_loss did not improve from 0.75082

# Epoch 00020: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-07.
# Epoch 21/120
# 1749/1749 [==============================] - 383s 219ms/step - loss: 0.7766 - f1: 0.3129 - val_loss: 0.7479 - val_f1: 0.3757

# Epoch 00021: val_loss improved from 0.75082 to 0.74793, saving model to ../cache/InceptionV3.h5
# Epoch 22/120
# 1749/1749 [==============================] - 383s 219ms/step - loss: 0.7778 - f1: 0.3123 - val_loss: 0.7495 - val_f1: 0.3689

# Epoch 00022: val_loss did not improve from 0.74793
# Epoch 23/120
# 1749/1749 [==============================] - 383s 219ms/step - loss: 0.7773 - f1: 0.3124 - val_loss: 0.7480 - val_f1: 0.3725

# Epoch 00023: val_loss did not improve from 0.74793
# Epoch 24/120
# 1749/1749 [==============================] - 382s 219ms/step - loss: 0.7740 - f1: 0.3146 - val_loss: 0.7491 - val_f1: 0.3715

# Epoch 00024: val_loss did not improve from 0.74793

# Epoch 00024: ReduceLROnPlateau reducing learning rate to 9.999999974752428e-08.
# Epoch 25/120
# 1749/1749 [==============================] - 382s 219ms/step - loss: 0.7737 - f1: 0.3153 - val_loss: 0.7473 - val_f1: 0.3761

# Epoch 00025: val_loss improved from 0.74793 to 0.74733, saving model to ../cache/InceptionV3.h5
# Epoch 26/120
# 1749/1749 [==============================] - 383s 219ms/step - loss: 0.7742 - f1: 0.3146 - val_loss: 0.7437 - val_f1: 0.3774

# Epoch 00026: val_loss improved from 0.74733 to 0.74374, saving model to ../cache/InceptionV3.h5
# Epoch 27/120
# 1749/1749 [==============================] - 383s 219ms/step - loss: 0.7743 - f1: 0.3141 - val_loss: 0.7483 - val_f1: 0.3721

# Epoch 00027: val_loss did not improve from 0.74374
# Epoch 28/120
# 1749/1749 [==============================] - 384s 219ms/step - loss: 0.7740 - f1: 0.3141 - val_loss: 0.7494 - val_f1: 0.3722

# Epoch 00028: val_loss did not improve from 0.74374
# Epoch 29/120
# 1749/1749 [==============================] - 383s 219ms/step - loss: 0.7741 - f1: 0.3147 - val_loss: 0.7469 - val_f1: 0.3752

# Epoch 00029: val_loss did not improve from 0.74374

# Epoch 00029: ReduceLROnPlateau reducing learning rate to 1.0000000116860975e-08.
# Epoch 30/120
# 1749/1749 [==============================] - 383s 219ms/step - loss: 0.7732 - f1: 0.3159 - val_loss: 0.7468 - val_f1: 0.3734

# Epoch 00030: val_loss did not improve from 0.74374
# Epoch 31/120
# 1749/1749 [==============================] - 384s 220ms/step - loss: 0.7745 - f1: 0.3138 - val_loss: 0.7500 - val_f1: 0.3702

# Epoch 00031: val_loss did not improve from 0.74374
# Epoch 32/120
# 1749/1749 [==============================] - 386s 221ms/step - loss: 0.7765 - f1: 0.3127 - val_loss: 0.7468 - val_f1: 0.3748
#   0%|          | 0/3099 [00:00<?, ?it/s]

# Epoch 00032: val_loss did not improve from 0.74374

# Epoch 00032: ReduceLROnPlateau reducing learning rate to 9.999999939225292e-10.
# 100%|██████████| 3099/3099 [02:47<00:00, 18.52it/s]
# 11702it [08:03, 24.19it/s]
# 27969 3103
# Epoch 1/2
# 1749/1749 [==============================] - 331s 189ms/step - loss: 1.1194 - f1: 0.0381 - val_loss: 1.1530 - val_f1: 0.0276
# Epoch 2/2
# 1749/1749 [==============================] - 328s 187ms/step - loss: 1.1066 - f1: 0.0477 - val_loss: 1.2052 - val_f1: 0.0364
# Epoch 1/120
# 1749/1749 [==============================] - 401s 229ms/step - loss: 1.0629 - f1: 0.0893 - val_loss: 1.0045 - val_f1: 0.1535

# Epoch 00001: val_loss improved from inf to 1.00452, saving model to ../cache/InceptionV3.h5
# Epoch 2/120
# 1749/1749 [==============================] - 380s 218ms/step - loss: 0.9884 - f1: 0.1612 - val_loss: 0.9702 - val_f1: 0.1899

# Epoch 00002: val_loss improved from 1.00452 to 0.97018, saving model to ../cache/InceptionV3.h5
# Epoch 3/120
# 1749/1749 [==============================] - 378s 216ms/step - loss: 0.9427 - f1: 0.2013 - val_loss: 0.8851 - val_f1: 0.2652

# Epoch 00003: val_loss improved from 0.97018 to 0.88508, saving model to ../cache/InceptionV3.h5
# Epoch 4/120
# 1749/1749 [==============================] - 379s 217ms/step - loss: 0.9145 - f1: 0.2240 - val_loss: 0.8796 - val_f1: 0.2782

# Epoch 00004: val_loss improved from 0.88508 to 0.87963, saving model to ../cache/InceptionV3.h5
# Epoch 5/120
#  978/1749 [===============>..............] - ETA: 2:24 - loss: 0.8989 - f1: 0.2330

In [23]:
np.save('../cache/oof_class_preds-14-1.npy', oof_class_preds)
np.save('../cache/sub_class_preds-14-1.npy', sub_class_preds)

In [24]:
# oof_class_preds = np.zeros((train_dataset_info.shape[0], 28))
# sub_class_preds = np.zeros((submit.shape[0], 28))
# score_predict = model.predict(image[np.newaxis])[0]
# oof_class_preds[idx] = score_predict

In [25]:
# for train_index, test_index in kf.split(indexes):
# ...    print("TRAIN:", train_index, "TEST:", test_index)

In [26]:
sub_class_preds.shape

(11702, 28)

In [27]:
predicted = []
for line in tqdm(sub_class_preds):
    label_predict = np.arange(28)[line>=0.2]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

100%|██████████| 11702/11702 [00:00<00:00, 99590.62it/s]


In [28]:
len(predicted)

11702

In [29]:
predicted

['2',
 '5 25',
 '0 5 25',
 '0 25',
 '0 7 25',
 '4',
 '0 4 25',
 '0 23 25',
 '0',
 '25',
 '17 18 25',
 '3 5',
 '0 2 25',
 '7 9 10 18 25',
 '23',
 '0 4 18',
 '2 14 25',
 '0 5',
 '14 21',
 '0 5',
 '6',
 '3 5',
 '0 11 16 17 25',
 '0 7',
 '0 4',
 '0 12 21 25',
 '0',
 '0',
 '0 5 25',
 '0',
 '0 13 21',
 '0 25',
 '14 16 17 18 21 25',
 '0 5 25',
 '0 7 25',
 '13',
 '0 25',
 '0 3',
 '0 5 21 25',
 '1',
 '0 16 17 25',
 '6 25',
 '0 21',
 '18 19 25',
 '0 14 16 25',
 '6',
 '0',
 '0 23',
 '0 6 11 23 25',
 '0',
 '0 16 17 25',
 '0 5',
 '20 23 26',
 '0 25',
 '3',
 '0 25',
 '0 17 25',
 '11 23',
 '0 25',
 '21 25',
 '2 22',
 '0 5 21 25',
 '2 14 16 25',
 '7 21',
 '23',
 '0 18 19 25',
 '3 6 21 25',
 '0 25',
 '0 1',
 '21 25',
 '2 3',
 '0 2',
 '14',
 '4',
 '21',
 '0',
 '2 4',
 '0 1',
 '0 25',
 '0 16 17 25',
 '6 25',
 '0 25',
 '0 21',
 '0 17 21 25',
 '16 17 18 19',
 '0 23',
 '20 23',
 '0 21',
 '14 16 17 25',
 '14',
 '0 25',
 '11 14',
 '23',
 '13',
 '0 16 17 25',
 '0 25',
 '7 16 17 18 25',
 '0 7 19 25',
 '24',
 '0

In [30]:
submit['Predicted'] = predicted

In [31]:
# Create submit
# submit = pd.read_csv('../data/sample_submission.csv')
# predicted = []
# draw_predict = []
# # model.load_weights('../cache/InceptionV3.h5')
# for name in tqdm(submit['Id']):
#     path = os.path.join('../data/test/', name)
#     image = data_generator.load_image(path, (SIZE,SIZE,3))/255.
#     score_predict = model.predict(image[np.newaxis])[0]
#     draw_predict.append(score_predict)
#     label_predict = np.arange(28)[score_predict>=0.2]
#     str_predict_label = ' '.join(str(l) for l in label_predict)
#     predicted.append(str_predict_label)

# submit['Predicted'] = predicted
# np.save('../cache/draw_predict_InceptionV3-8.npy', score_predict)
# submit.to_csv('../submissions/submit_InceptionV3.csv', index=False)

In [32]:
submit.to_csv('../submissions/sub14-a.csv', index=False)

In [33]:
#https://stackoverflow.com/questions/1855095/how-to-create-a-zip-archive-of-a-directory
def backup_project_as_zip(project_dir, zip_file):
    assert(os.path.isdir(project_dir))
    assert(os.path.isdir(os.path.dirname(zip_file)))
    shutil.make_archive(zip_file.replace('.zip',''), 'zip', project_dir)
    pass

In [34]:
import datetime, shutil
now = datetime.datetime.now()
print(now)
PROJECT_PATH = '/home/watts/lal/Kaggle/kagglehp/scripts_nbs'
backup_project_as_zip(PROJECT_PATH, '../cache/code.scripts_nbs.%s.zip'%now)

2018-10-30 22:02:00.026549


In [35]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub14-a.csv -m ""

100%|████████████████████████████████████████| 492k/492k [00:12<00:00, 40.0kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationCPU times: user 336 ms, sys: 153 ms, total: 488 ms
Wall time: 15.6 s


In [36]:
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub14-a.csv   2018-10-30 16:32:37               complete  0.452        None          
sub13-c.csv   2018-10-29 19:24:00               complete  0.459        None          
sub13-bb.csv  2018-10-29 19:23:01               complete  0.454        None          
sub13-b.csv   2018-10-29 19:21:53               complete  0.448        None          
sub13-a.csv   2018-10-29 19:20:40               complete  0.444        None          
sub12-d.csv   2018-10-26 02:09:32               complete  0.466        None          
sub12-h.csv   2018-10-26 02:07:56               complete  0.389        None          
sub12-g.csv   2018-10-25 00:55:10               complete  0.433        None          
sub12-c.csv   2018-10-25 00:45:32               complete  0.469        None          
sub12-bb.csv  2018-10-25 00:43:34               comple

In [37]:
predicted = []
for line in tqdm(sub_class_preds):
    label_predict = np.arange(28)[line>=0.25]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)

100%|██████████| 11702/11702 [00:00<00:00, 98690.10it/s]


In [38]:
submit['Predicted'] = predicted
submit.to_csv('../submissions/sub14-b.csv', index=False)

In [39]:
import datetime, shutil
now = datetime.datetime.now()
print(now)
PROJECT_PATH = '/home/watts/lal/Kaggle/kagglehp/scripts_nbs'
backup_project_as_zip(PROJECT_PATH, '../cache/code.scripts_nbs.%s.zip'%now)

2018-10-30 22:03:04.219937


In [40]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub14-b.csv -m ""

100%|████████████████████████████████████████| 486k/486k [00:12<00:00, 39.7kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationCPU times: user 322 ms, sys: 168 ms, total: 491 ms
Wall time: 15.2 s


In [41]:
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub14-b.csv   2018-10-30 16:33:35               complete  0.459        None          
sub14-a.csv   2018-10-30 16:32:37               complete  0.452        None          
sub13-c.csv   2018-10-29 19:24:00               complete  0.459        None          
sub13-bb.csv  2018-10-29 19:23:01               complete  0.454        None          
sub13-b.csv   2018-10-29 19:21:53               complete  0.448        None          
sub13-a.csv   2018-10-29 19:20:40               complete  0.444        None          
sub12-d.csv   2018-10-26 02:09:32               complete  0.466        None          
sub12-h.csv   2018-10-26 02:07:56               complete  0.389        None          
sub12-g.csv   2018-10-25 00:55:10               complete  0.433        None          
sub12-c.csv   2018-10-25 00:45:32               comple

In [42]:
d = {0.3:'bb', 0.35:'c', 0.4:'d', 0.45:'e', 0.5:'f'}

In [43]:
for alpha in [0.3, 0.35, 0.4, 0.45, 0.5]:
    predicted = []
    for line in tqdm(sub_class_preds):
        label_predict = np.arange(28)[line>=alpha]
        str_predict_label = ' '.join(str(l) for l in label_predict)
        predicted.append(str_predict_label)
    submit['Predicted'] = predicted
    name = '../submissions/sub14-' + d[alpha] + '.csv'
    print(name)
    submit.to_csv(name, index=False)

  0%|          | 0/11702 [00:00<?, ?it/s]

../submissions/sub14-bb.csv
../submissions/sub14-c.csv


  0%|          | 0/11702 [00:00<?, ?it/s]

../submissions/sub14-d.csv
../submissions/sub14-e.csv


100%|██████████| 11702/11702 [00:00<00:00, 127874.32it/s]

../submissions/sub14-f.csv


In [44]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub14-bb.csv -m ""

100%|████████████████████████████████████████| 482k/482k [00:08<00:00, 51.7kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationCPU times: user 362 ms, sys: 160 ms, total: 522 ms
Wall time: 16.4 s


In [45]:
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub14-bb.csv  2018-10-30 16:34:35               complete  0.461        None          
sub14-b.csv   2018-10-30 16:33:35               complete  0.459        None          
sub14-a.csv   2018-10-30 16:32:37               complete  0.452        None          
sub13-c.csv   2018-10-29 19:24:00               complete  0.459        None          
sub13-bb.csv  2018-10-29 19:23:01               complete  0.454        None          
sub13-b.csv   2018-10-29 19:21:53               complete  0.448        None          
sub13-a.csv   2018-10-29 19:20:40               complete  0.444        None          
sub12-d.csv   2018-10-26 02:09:32               complete  0.466        None          
sub12-h.csv   2018-10-26 02:07:56               complete  0.389        None          
sub12-g.csv   2018-10-25 00:55:10               comple

In [46]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub14-c.csv -m ""
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 478k/478k [00:08<00:00, 47.9kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub14-c.csv   2018-10-30 16:35:09               complete  0.466        None          
sub14-bb.csv  2018-10-30 16:34:35               complete  0.461        None          
sub14-b.csv   2018-10-30 16:33:35               complete  0.459        None          
sub14-a.csv   2018-10-30 16:32:37               complete  0.452        None          
sub13-c.csv   2018-10-29 19:24:00               complete  0.459        None          
sub13-bb.csv  2018-10-29 19:23:01               complete  0.454        None          
sub13-b.csv   2018-10-29 19:21:53               complete  0.448        None          
sub13-a.csv   2018-10-29 19:20:40               complete  0.444        None    

In [47]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub14-d.csv -m ""
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 474k/474k [00:12<00:00, 39.1kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub14-d.csv   2018-10-30 16:36:00               complete  0.464        None          
sub14-c.csv   2018-10-30 16:35:09               complete  0.466        None          
sub14-bb.csv  2018-10-30 16:34:35               complete  0.461        None          
sub14-b.csv   2018-10-30 16:33:35               complete  0.459        None          
sub14-a.csv   2018-10-30 16:32:37               complete  0.452        None          
sub13-c.csv   2018-10-29 19:24:00               complete  0.459        None          
sub13-bb.csv  2018-10-29 19:23:01               complete  0.454        None          
sub13-b.csv   2018-10-29 19:21:53               complete  0.448        None    

In [46]:
oof_class_preds.shape

(31072, 28)

In [47]:
data.head()

,Id,Target
0,00070df0-bbc3-11e8-b2bc-ac1f6b6435d0,16 0
1,000a6c98-bb9b-11e8-b2b9-ac1f6b6435d0,7 1 2 0
2,000a9596-bbc4-11e8-b2bc-ac1f6b6435d0,5
3,000c99ba-bba4-11e8-b2b9-ac1f6b6435d0,1
4,001838f8-bbca-11e8-b2bc-ac1f6b6435d0,18


In [48]:
data.shape

(31072, 2)

In [49]:
n_classes = 28
y_train = np.zeros(oof_class_preds.shape)
y_train.shape

(31072, 28)

In [50]:
idx = 0
for labels in tqdm(data['Target'].str.split(' ')):
#     print(labels)
    for label in labels:
        y_train[idx][int(label)] = 1
    idx += 1

100%|██████████| 31072/31072 [00:00<00:00, 741901.66it/s]


In [51]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.]])

In [52]:
y_train[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [54]:
sub_ridge_preds = np.zeros(sub_class_preds.shape)
from sklearn.linear_model import Ridge
for cls in np.arange(n_classes):
    for alpha in [0.001, 0.01, 0.1, 0.5, 0.75, 1.0]:
        y = y_train[:, cls]
        X = oof_class_preds
        clf = Ridge(alpha=alpha)
        clf.fit(X,y)
        score_ = clf.score(X, y)
        print(alpha, score_)
    print('------------------')
#         X_test = sub_class_preds[:, cls]
#         preds_ = clf.predict(X_test)
#         sub_ridge_preds[:,cls] = preds_

0.001 0.6174081518752018
0.01 0.6174081494907646
0.1 0.6174079419635514
0.5 0.6174049197331472
0.75 0.6174023910597577
1.0 0.6173997536697944
------------------
0.001 0.7042131450176463
0.01 0.7042131449021258
0.1 0.7042131341504188
0.5 0.7042129258072632
0.75 0.7042126913081677
1.0 0.7042123865546952
------------------
0.001 0.6405293650278552
0.01 0.6405293643199262
0.1 0.6405293035770623
0.5 0.6405284696126301
0.75 0.6405278129689744
1.0 0.6405271539313317
------------------
0.001 0.5206868082561187
0.01 0.5206868064937693
0.1 0.5206866550090807
0.5 0.5206845661735036
0.75 0.5206829204240566
1.0 0.5206812723936773
------------------
0.001 0.6225629182679313
0.01 0.6225628870674288
0.1 0.6225602493284829
0.5 0.6225265347174933
0.75 0.6225024258166685
1.0 0.6224802007531789
------------------
0.001 0.47625033808763034
0.01 0.4762503310360723
0.1 0.47624973372408597
0.5 0.4762420258070793
0.75 0.4762364393569697
1.0 0.47623122503579396
------------------
0.001 0.37015383088158105
0.01 

In [55]:
sub_ridge_preds = np.zeros(sub_class_preds.shape)
from sklearn.linear_model import Ridge
for cls in np.arange(n_classes):
    y = y_train[:, cls]
    X = oof_class_preds
    clf = Ridge(alpha=0.1)
    clf.fit(X,y)
    X_test = sub_class_preds
    preds_ = clf.predict(X_test)
    sub_ridge_preds[:,cls] = preds_

In [56]:
sub_class_preds

array([[9.38001867e-02, 1.59430779e-04, 9.98871672e-01, ...,
        1.77784316e-03, 1.12266340e-04, 5.20836841e-09],
       [2.47062426e-02, 2.68127583e-04, 7.85995722e-04, ...,
        6.51888692e-01, 8.71524611e-04, 3.26655725e-05],
       [8.41529155e-01, 2.72278007e-04, 3.96186303e-03, ...,
        9.23864961e-01, 2.01543609e-03, 1.92988443e-05],
       ...,
       [6.59056642e-04, 5.12143007e-05, 3.49444263e-05, ...,
        1.77463120e-03, 5.20864920e-08, 5.91902866e-09],
       [5.01914832e-01, 9.99162483e-01, 2.73049554e-03, ...,
        1.16331837e-02, 1.53418808e-04, 1.20560289e-06],
       [5.07521251e-01, 3.52088286e-04, 3.01849514e-03, ...,
        6.92711103e-01, 1.99362053e-03, 1.05607675e-06]])

In [57]:
np.arange(n_classes)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])

In [58]:
sub_class_preds[:, 0].shape

(11702,)

In [59]:
sub_ridge_preds

array([[ 1.26424064e-01,  8.84581981e-03,  8.85833838e-01, ...,
         3.20592330e-02,  4.54385428e-03, -6.92529375e-05],
       [ 5.51528260e-02,  2.13827355e-03,  1.63897473e-02, ...,
         5.38178358e-01,  1.06812025e-03,  1.39571143e-03],
       [ 7.77233272e-01, -1.67743788e-03,  1.87791802e-02, ...,
         7.51411388e-01, -1.45306321e-03,  1.27131495e-03],
       ...,
       [ 2.62432560e-02,  4.66381707e-04,  7.97892029e-03, ...,
         2.92185935e-02,  3.12140745e-03,  1.76712611e-04],
       [ 4.95625470e-01,  8.71181686e-01,  1.99755452e-02, ...,
         3.70837162e-02,  3.03441473e-03,  3.03600332e-05],
       [ 4.96285545e-01,  1.04218405e-02,  2.67882221e-02, ...,
         5.85205684e-01,  4.37290352e-03,  1.52204091e-06]])

In [60]:
predicted = []
alpha = 0.35
for line in tqdm(sub_ridge_preds):
    label_predict = np.arange(28)[line>=alpha]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit['Predicted'] = predicted
name = '../submissions/sub12-g.csv'
print(name)
submit.to_csv(name, index=False)

100%|██████████| 11702/11702 [00:00<00:00, 88242.24it/s]

../submissions/sub12-g.csv


In [61]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub12-g.csv -m ""

Successfully submitted to Human Protein Atlas Image ClassificationCPU times: user 372 ms, sys: 201 ms, total: 573 ms
Wall time: 17.3 s


In [62]:
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub12-g.csv   2018-10-25 00:55:10               complete  0.433        None          
sub12-c.csv   2018-10-25 00:45:32               complete  0.469        None          
sub12-bb.csv  2018-10-25 00:43:34               complete  0.466        None          
sub12-b.csv   2018-10-25 00:41:50               complete  0.457        None          
sub12-a.csv   2018-10-25 00:40:56               complete  0.449        None          
sub11-k.csv   2018-10-24 00:35:39               complete  0.346        None          
sub11-j.csv   2018-10-24 00:34:46               complete  0.366        None          
sub11-j.csv   2018-10-24 00:33:17               complete  0.000        None          
sub11-i.csv   2018-10-24 00:24:24               complete  0.389        None          
sub11-h.csv   2018-10-24 00:21:18               comple

In [63]:
predicted = []
alpha = 0.4
for line in tqdm(sub_ridge_preds):
    label_predict = np.arange(28)[line>=alpha]
    str_predict_label = ' '.join(str(l) for l in label_predict)
    predicted.append(str_predict_label)
submit['Predicted'] = predicted
name = '../submissions/sub12-h.csv'
print(name)
submit.to_csv(name, index=False)

100%|██████████| 11702/11702 [00:00<00:00, 79991.76it/s]

../submissions/sub12-h.csv


In [64]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub12-h.csv -m ""
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

Successfully submitted to Human Protein Atlas Image ClassificationWarning: Looks like you're using an outdated API Version, please consider updating (server 1.5.0 / client 1.3.8)
fileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub12-h.csv   2018-10-26 02:07:56               complete  0.389        None          
sub12-g.csv   2018-10-25 00:55:10               complete  0.433        None          
sub12-c.csv   2018-10-25 00:45:32               complete  0.469        None          
sub12-bb.csv  2018-10-25 00:43:34               complete  0.466        None          
sub12-b.csv   2018-10-25 00:41:50               complete  0.457        None          
sub12-a.csv   2018-10-25 00:40:56               complete  0.449        None          
sub11-k.csv   2018-10-24 00:35:39               complete  0.346        None          
sub11-j.csv   2018-10-24 00:34:46              

In [65]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub12-d.csv -m ""
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

Successfully submitted to Human Protein Atlas Image ClassificationWarning: Looks like you're using an outdated API Version, please consider updating (server 1.5.0 / client 1.3.8)
fileName      date                 description  status    publicScore  privateScore  
------------  -------------------  -----------  --------  -----------  ------------  
sub12-d.csv   2018-10-26 02:09:32               complete  0.466        None          
sub12-h.csv   2018-10-26 02:07:56               complete  0.389        None          
sub12-g.csv   2018-10-25 00:55:10               complete  0.433        None          
sub12-c.csv   2018-10-25 00:45:32               complete  0.469        None          
sub12-bb.csv  2018-10-25 00:43:34               complete  0.466        None          
sub12-b.csv   2018-10-25 00:41:50               complete  0.457        None          
sub12-a.csv   2018-10-25 00:40:56               complete  0.449        None          
sub11-k.csv   2018-10-24 00:35:39              

In [1]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub61-max-rnxt-05.csv -m ""
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 480k/480k [00:12<00:00, 39.2kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName               date                 description  status    publicScore  privateScore  
---------------------  -------------------  -----------  --------  -----------  ------------  
sub61-max-rnxt-05.csv  2019-01-05 14:52:48               complete  0.545        None          
sub59-max-045-1.csv    2019-01-01 09:01:41               complete  0.520        None          
sub59-max-05-1.csv     2019-01-01 08:59:47               complete  0.500        None          
sub50-max-045-l.csv    2019-01-01 05:15:38               complete  0.569        None          
sub59-max-040.csv      2019-01-01 05:12:22               complete  0.530        None          
sub59-max-045.csv      2018-12-31 08:45:31               complete  0.545        None          
sub59-max-049.csv      2018-12-31 08:44:03               complete  0.535        None         

In [4]:
%%time
# !kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub61-max-rnxt-045.csv -m ""
# from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName                date                 description  status    publicScore  privateScore  
----------------------  -------------------  -----------  --------  -----------  ------------  
sub61-max-rnxt-045.csv  2019-01-05 14:53:42               complete  0.538        None          
sub61-max-rnxt-05.csv   2019-01-05 14:52:48               complete  0.545        None          
sub59-max-045-1.csv     2019-01-01 09:01:41               complete  0.520        None          
sub59-max-05-1.csv      2019-01-01 08:59:47               complete  0.500        None          
sub50-max-045-l.csv     2019-01-01 05:15:38               complete  0.569        None          
sub59-max-040.csv       2019-01-01 05:12:22               complete  0.530        None          
sub59-max-045.csv       2018-12-31 08:45:31               complete  0.545        None          
sub59-max-049.csv       2018-12-31 08:44:03               complete  0.535        None          
sub59-max-051.csv       2018-12-31 08:43

In [5]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub61-max-rnxt-055.csv -m ""
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 473k/473k [00:08<00:00, 42.6kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName                date                 description  status    publicScore  privateScore  
----------------------  -------------------  -----------  --------  -----------  ------------  
sub61-max-rnxt-055.csv  2019-01-05 14:55:11               complete  0.539        None          
sub61-max-rnxt-045.csv  2019-01-05 14:53:42               complete  0.538        None          
sub61-max-rnxt-05.csv   2019-01-05 14:52:48               complete  0.545        None          
sub59-max-045-1.csv     2019-01-01 09:01:41               complete  0.520        None          
sub59-max-05-1.csv      2019-01-01 08:59:47               complete  0.500        None          
sub50-max-045-l.csv     2019-01-01 05:15:38               complete  0.569        None          
sub59-max-040.csv       2019-01-01 05:12:22               complete  0.530        None

In [7]:
import pandas as pd
leak_test = pd.read_csv('../cache/leak_test.csv')
sub50 = pd.read_csv('../submissions/sub61-max-rnxt-05.csv')
for idx, row in leak_test.iterrows():
    test_id = row['Id']
    target = row['Target']
    sub50.loc[sub50.Id == test_id, 'Predicted'] = target
sub50.to_csv('../submissions/sub61-max-rnxt-05-l.csv', index=False)

In [8]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub61-max-rnxt-05-l.csv -m ""
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 480k/480k [00:12<00:00, 39.8kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName                 date                 description  status    publicScore  privateScore  
-----------------------  -------------------  -----------  --------  -----------  ------------  
sub61-max-rnxt-05-l.csv  2019-01-05 14:58:56               complete  0.548        None          
sub61-max-rnxt-055.csv   2019-01-05 14:55:11               complete  0.539        None          
sub61-max-rnxt-045.csv   2019-01-05 14:53:42               complete  0.538        None          
sub61-max-rnxt-05.csv    2019-01-05 14:52:48               complete  0.545        None          
sub59-max-045-1.csv      2019-01-01 09:01:41               complete  0.520        None          
sub59-max-05-1.csv       2019-01-01 08:59:47               complete  0.500        None          
sub50-max-045-l.csv      2019-01-01 05:15:38               complete  0.569   

In [9]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub61-max-rnxt-035.csv -m ""
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 511k/511k [00:13<00:00, 40.0kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName                 date                 description  status    publicScore  privateScore  
-----------------------  -------------------  -----------  --------  -----------  ------------  
sub61-max-rnxt-035.csv   2019-01-05 20:09:43  None         complete  0.516        None          
sub61-max-rnxt-05-l.csv  2019-01-05 14:58:56               complete  0.548        None          
sub61-max-rnxt-055.csv   2019-01-05 14:55:11               complete  0.539        None          
sub61-max-rnxt-045.csv   2019-01-05 14:53:42               complete  0.538        None          
sub61-max-rnxt-05.csv    2019-01-05 14:52:48               complete  0.545        None          
sub59-max-045-1.csv      2019-01-01 09:01:41               complete  0.520        None          
sub59-max-05-1.csv       2019-01-01 08:59:47               complete  0.500   

In [12]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub61-max-wrn-05.csv -m ""
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName                       date                 description  status    publicScore  privateScore  
-----------------------------  -------------------  -----------  --------  -----------  ------------  
sub61-max-wrn-05.csv           2019-01-08 04:10:11               complete  0.501        None          
sub61-max-rnxt50-dn121-05.csv  2019-01-07 05:05:35               complete  0.210        None          
sub61-max-dn121-05-l.csv       2019-01-07 04:55:46               complete  0.577        None          
sub61-max-dn121-045.csv        2019-01-07 04:44:56               complete  0.556        None          
sub61-max-dn121-051.csv        2019-01-07 04:44:15               complete  0.560        None          
sub61-max-dn121-05.csv         2019-01-07 04:42:53               complete  0.561        None          
sub61-max-rnxt-035.csv         2019-01-05 20:09:43  None         complete  0.516        None          
sub61-max-rnxt-05-l.csv        2019-01-05 14:58:56               complete

In [16]:
# %%time
# !kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub61-max-wrn-045.csv -m ""
# from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

fileName                       date                 description  status    publicScore  privateScore  
-----------------------------  -------------------  -----------  --------  -----------  ------------  
sub61-max-wrn-045.csv          2019-01-08 04:11:56               complete  0.494        None          
sub61-max-wrn-05.csv           2019-01-08 04:10:11               complete  0.501        None          
sub61-max-rnxt50-dn121-05.csv  2019-01-07 05:05:35               complete  0.210        None          
sub61-max-dn121-05-l.csv       2019-01-07 04:55:46               complete  0.577        None          
sub61-max-dn121-045.csv        2019-01-07 04:44:56               complete  0.556        None          
sub61-max-dn121-051.csv        2019-01-07 04:44:15               complete  0.560        None          
sub61-max-dn121-05.csv         2019-01-07 04:42:53               complete  0.561        None          
sub61-max-rnxt-035.csv         2019-01-05 20:09:43  None         complete

In [17]:
%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub61-max-wrn-055.csv -m ""
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

100%|████████████████████████████████████████| 472k/472k [00:12<00:00, 38.2kB/s]
Successfully submitted to Human Protein Atlas Image ClassificationfileName                       date                 description  status    publicScore  privateScore  
-----------------------------  -------------------  -----------  --------  -----------  ------------  
sub61-max-wrn-055.csv          2019-01-08 04:20:23               complete  0.493        None          
sub61-max-wrn-045.csv          2019-01-08 04:11:56               complete  0.494        None          
sub61-max-wrn-05.csv           2019-01-08 04:10:11               complete  0.501        None          
sub61-max-rnxt50-dn121-05.csv  2019-01-07 05:05:35               complete  0.210        None          
sub61-max-dn121-05-l.csv       2019-01-07 04:55:46               complete  0.577        None          
sub61-max-dn121-045.csv        2019-01-07 04:44:56               complete  0.556        None          
sub61-max-dn121-051.csv      

In [18]:

%%time
!kaggle competitions submit -c human-protein-atlas-image-classification -f ../submissions/sub61-max-f-dn121.csv -m ""
from time import sleep
sleep(10)
!kaggle competitions submissions -c human-protein-atlas-image-classification

Traceback (most recent call last):
  File "/home/watts/anaconda3/envs/hpg/bin/kaggle", line 11, in <module>
    sys.exit(main())
  File "/home/watts/anaconda3/envs/hpg/lib/python3.6/site-packages/kaggle/cli.py", line 50, in main
    out = args.func(**command_args)
  File "/home/watts/anaconda3/envs/hpg/lib/python3.6/site-packages/kaggle/api/kaggle_api_extended.py", line 515, in competition_submit_cli
    competition, quiet)
  File "/home/watts/anaconda3/envs/hpg/lib/python3.6/site-packages/kaggle/api/kaggle_api_extended.py", line 466, in competition_submit
    content_length=os.path.getsize(file_name),
  File "/home/watts/anaconda3/envs/hpg/lib/python3.6/genericpath.py", line 50, in getsize
    return os.stat(filename).st_size
FileNotFoundError: [Errno 2] No such file or directory: '../submissions/sub61-max-f-dn121.csv'
fileName                       date                 description  status    publicScore  privateScore  
-----------------------------  -------------------  -----------  

In [21]:
import numpy as np
pred_1 = np.load('../cache/sub_class_preds-61-rnxt50.npy')
pred_2 = np.load('../cache/sub_class_preds-61-dn121.npy')

In [22]:
pred_3 = ((0.545*pred_1) + (0.561*pred_2))/(0.545 + 0.561) 

In [1]:
import numpy as np
t1 = np.random.normal(0,0.1,(11702,28))

In [2]:
t1

array([[-0.18883886, -0.10663436, -0.0341205 , ..., -0.10723461,
        -0.04620481,  0.00371242],
       [ 0.09980148, -0.04471247,  0.04088159, ...,  0.01994856,
         0.06292838, -0.02976964],
       [ 0.05861086,  0.13998308, -0.13870705, ...,  0.05607048,
        -0.08925387,  0.14337828],
       ...,
       [ 0.06233832,  0.16623217, -0.02785105, ..., -0.04381846,
         0.10264066, -0.07743156],
       [-0.09326148, -0.04003283, -0.12436623, ...,  0.03500652,
         0.17534864, -0.08211362],
       [ 0.04677171, -0.05864172,  0.10129688, ..., -0.02413385,
        -0.0430419 ,  0.16270086]])

In [3]:
np.save('../cache/t1', t1)

In [4]:
t2 = np.load('../cache/t1.npy')

In [5]:
t2

array([[-0.18883886, -0.10663436, -0.0341205 , ..., -0.10723461,
        -0.04620481,  0.00371242],
       [ 0.09980148, -0.04471247,  0.04088159, ...,  0.01994856,
         0.06292838, -0.02976964],
       [ 0.05861086,  0.13998308, -0.13870705, ...,  0.05607048,
        -0.08925387,  0.14337828],
       ...,
       [ 0.06233832,  0.16623217, -0.02785105, ..., -0.04381846,
         0.10264066, -0.07743156],
       [-0.09326148, -0.04003283, -0.12436623, ...,  0.03500652,
         0.17534864, -0.08211362],
       [ 0.04677171, -0.05864172,  0.10129688, ..., -0.02413385,
        -0.0430419 ,  0.16270086]])

In [7]:
sum(t1 == t2)

array([11702, 11702, 11702, 11702, 11702, 11702, 11702, 11702, 11702,
       11702, 11702, 11702, 11702, 11702, 11702, 11702, 11702, 11702,
       11702, 11702, 11702, 11702, 11702, 11702, 11702, 11702, 11702,
       11702])

In [9]:
import os
TEST='../data/test'
test_names = list({f[:36] for f in os.listdir(TEST)})
test_names

['a1c8b614-bad0-11e8-b2b8-ac1f6b6435d0',
 'fb3c40e8-bac7-11e8-b2b7-ac1f6b6435d0',
 'fad8813e-bad1-11e8-b2b8-ac1f6b6435d0',
 '6fc291fe-bad9-11e8-b2b9-ac1f6b6435d0',
 'e8c2d8f8-bad8-11e8-b2b9-ac1f6b6435d0',
 'a685ded0-bad4-11e8-b2b8-ac1f6b6435d0',
 '2e4a7f3c-bad6-11e8-b2b9-ac1f6b6435d0',
 'acedd882-bad3-11e8-b2b8-ac1f6b6435d0',
 'f80a0d36-bad8-11e8-b2b9-ac1f6b6435d0',
 'ea9e6a32-bac6-11e8-b2b7-ac1f6b6435d0',
 '3e163322-bad9-11e8-b2b9-ac1f6b6435d0',
 '65da048a-bad7-11e8-b2b9-ac1f6b6435d0',
 '9d5eeae0-baca-11e8-b2b8-ac1f6b6435d0',
 '563a3ed2-bacd-11e8-b2b8-ac1f6b6435d0',
 'f1826eec-bac7-11e8-b2b7-ac1f6b6435d0',
 '2e28e03a-bad5-11e8-b2b8-ac1f6b6435d0',
 'd7094a06-bac5-11e8-b2b7-ac1f6b6435d0',
 'd1f188f4-bad3-11e8-b2b8-ac1f6b6435d0',
 '20e98dce-badb-11e8-b2b9-ac1f6b6435d0',
 '8fe5790a-bada-11e8-b2b9-ac1f6b6435d0',
 '25ed1756-bad5-11e8-b2b8-ac1f6b6435d0',
 'fcb8e0b6-bad6-11e8-b2b9-ac1f6b6435d0',
 '87c336c8-baca-11e8-b2b8-ac1f6b6435d0',
 '1863568a-bad6-11e8-b2b9-ac1f6b6435d0',
 'd1edfe9c-bad2-